In [ ]:
import pandas as pd
from natsort import natsorted
import streamlit as st

# GitHub raw file URLs
elective_file_url = 'https://raw.githubusercontent.com/satyam26en/SECTION/main/elective%20-%20Sheet1%20(1).csv'
core_file_url = 'https://raw.githubusercontent.com/satyam26en/SECTION/main/core_section%20-%20Sheet1.csv'

# Load the CSV files from GitHub
elective_df = pd.read_csv(elective_file_url)
core_df = pd.read_csv(core_file_url)

# Rename columns for better clarity
elective_df.columns = elective_df.iloc[0]
elective_df = elective_df[1:]

core_df.columns = core_df.iloc[0]
core_df = core_df[1:]

# Merge the dataframes on 'Roll No.'
merged_df = pd.merge(elective_df, core_df, on='Roll No.', how='outer')

# Function to filter and process the dataframe
def filter_and_process_data(merged_df):
    # Split the elective sections into two columns
    merged_df['Professional Elective 1'] = merged_df.groupby('Roll No.')['Elective section'].transform(lambda x: x.iloc[0])
    merged_df['Professional Elective 2'] = merged_df.groupby('Roll No.')['Elective section'].transform(lambda x: x.iloc[1] if len(x) > 1 else None)

    # Drop duplicates and unnecessary columns
    merged_df = merged_df[['Roll No.', 'Professional Elective 1', 'Professional Elective 2', 'Core Section']].drop_duplicates()

    # Function to rearrange elective sections
    def rearrange_electives(row):
        if row['Professional Elective 1'] and row['Professional Elective 1'].startswith(('HPC', 'DOS')):
            row['Professional Elective 1'], row['Professional Elective 2'] = row['Professional Elective 2'], row['Professional Elective 1']
        return row

    # Apply the function to each row in the dataframe
    merged_df = merged_df.apply(rearrange_electives, axis=1)

    # Sort the dataframe by 'Roll No.' and return
    return merged_df.sort_values(by='Roll No.').reset_index(drop=True)

# Process the data
processed_df = filter_and_process_data(merged_df)

# Function to get details for a specific section
def get_details_by_section(processed_df, section, section_type='core'):
    if section_type == 'core':
        result_df = processed_df[processed_df['Core Section'] == section]
    else:
        result_df = processed_df[(processed_df['Professional Elective 1'] == section) | (processed_df['Professional Elective 2'] == section)]
    return result_df[['Roll No.', 'Core Section', 'Professional Elective 1', 'Professional Elective 2']]

# Function to get details of a specific roll number
def get_details_by_roll_number(processed_df, roll_number):
    result_df = processed_df[processed_df['Roll No.'].astype(int) == roll_number]
    return result_df[['Roll No.', 'Core Section', 'Professional Elective 1', 'Professional Elective 2']]

# Get unique section lists and sort them naturally
core_sections = natsorted(processed_df['Core Section'].dropna().unique())
prof_elective_1_sections = natsorted(processed_df['Professional Elective 1'].dropna().unique())
prof_elective_2_sections = natsorted(processed_df['Professional Elective 2'].dropna().unique())

# Streamlit UI
st.title('Student Information System')

query_section = st.radio("Do you want to know the roll list of any particular section?", ('yes', 'no'))
if query_section == 'yes':
    section_type = st.radio("Do you want the details of core section or professional elective?", ('core', 'professional'))

    if section_type == 'core':
        section_name = st.selectbox("Select a core section:", core_sections)
    else:
        elective_type = st.radio("Do you want the details of professional elective 1 or professional elective 2?", ('professional elective 1', 'professional elective 2'))
        if elective_type == 'professional elective 1':
            section_name = st.selectbox("Select a professional elective 1 section:", prof_elective_1_sections)
        else:
            section_name = st.selectbox("Select a professional elective 2 section:", prof_elective_2_sections)
        section_type = elective_type

    section_result_df = get_details_by_section(processed_df, section_name, section_type)

    st.write(f"Roll numbers and elective details for {section_name} {section_type}:")
    st.dataframe(section_result_df.reset_index(drop=True))

    csv = section_result_df.to_csv(index=False)
    st.download_button(label="Download results as CSV", data=csv, file_name=f'{section_name}_{section_type}_details.csv', mime='text/csv')
else:
    specific_roll = st.radio("Do you want to know the information of any particular roll number?", ('yes', 'no'))
    if specific_roll == 'yes':
        roll_number = st.number_input("Enter the roll number:", min_value=0, step=1)
        if st.button("Get details"):
            roll_number_result_df = get_details_by_roll_number(processed_df, roll_number)
            st.write(f"Details for roll number {roll_number}:")
            st.dataframe(roll_number_result_df.reset_index(drop=True))

            csv = roll_number_result_df.to_csv(index=False)
            st.download_button(label="Download roll number details as CSV", data=csv, file_name=f'roll_number_{roll_number}_details.csv', mime='text/csv')
    else:
        start_range = st.number_input("Enter the start of the roll number range:", min_value=0, step=1)
        end_range = st.number_input("Enter the end of the roll number range:", min_value=0, step=1)

        if st.button("Get range details"):
            range_result_df = processed_df[(processed_df['Roll No.'].astype(int) >= start_range) & (processed_df['Roll No.'].astype(int) <= end_range)]
            st.write("Processed Data:")
            st.dataframe(range_result_df[['Roll No.', 'Core Section', 'Professional Elective 1', 'Professional Elective 2']].reset_index(drop=True))

            csv = range_result_df.to_csv(index=False)
            st.download_button(label="Download range details as CSV", data=csv, file_name='filtered_data.csv', mime='text/csv')
